In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
from itertools import chain
import os
import random
import zipfile
!pip install --upgrade pip 
!pip install efficientnet_pytorch
!pip install timm
!pip install linformer
!pip install vit-pytorch
# Pytorch Implementation from this Github: # https://github.com/lukemelas/EfficientNet-PyTorch  
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim 
import time 
import torchvision 
import matplotlib.pyplot as plt 
import copy 

from torch.optim import lr_scheduler
from linformer import Linformer
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader 
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from skimage import io, transform 
from sklearn.model_selection import train_test_split 
from tqdm.notebook import tqdm
from vit_pytorch.efficient import ViT
from vit_pytorch.distill import DistillableViT, DistillWrapper
from torchvision.models import resnet50

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install efficientnet_pytorch
#from efficientnet_pytorch import EfficientNet
import timm

In [ ]:
# Training settings
batch_size = 64
epochs = 11
lr = 3e-5
gamma = 0.7
seed = 42
device = 'cuda'

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [ ]:
# Loading data 
dataset_dir = "../input/cassava-leaf-disease-classification/"
data_df = pd.read_csv(dataset_dir + "train.csv")   

# Add to column Image_ID the image path in dataframe 
data_df["path"] = dataset_dir + "train_images/" + data_df["image_id"] 

# Rearrange column order
data_df = data_df[["image_id", "path", "label"]]

In [ ]:
pd.set_option('display.max_colwidth', -1)
data_df.head() 

In [ ]:
# Look how an image looks like

img = data_df.iloc[0] 
img = Image.open(img["path"]) # e.g. open image of picture
img 

In [ ]:
# Preprocessing step 
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224), 
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229, 0.224,0.225))
]) 

val_transform = transforms.Compose([
    transforms.Resize(256), 
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


In [ ]:
# split data in train and validation data 
train_list, valid_list = train_test_split(data_df, test_size=0.15) 

In [ ]:
# Create custom dataset with tensor for each image and label for each image
# source: https://stackoverflow.com/questions/61391919/loading-image-data-from-pandas-to-pytorch 

class MyDataset(Dataset): 
    def __init__(self, dataframe, transform = None): 
        self.dataframe = dataframe 
        self.transform = transform
        
    def __len__(self): 
        return len(self.dataframe) 
    
    def __getitem__(self, index): 
        row = self.dataframe.iloc[index] 
        img = Image.open(row["path"])  
        #tensor = torchvision.transforms.functional.to_tensor(img)
        label = row["label"]
        
        if self.transform: 
            img = self.transform(img)
        return img, label
    
        
# create dataset with tensors and their belonging labels for each image
train_data = MyDataset(train_list, transform = train_transform) 
valid_data = MyDataset(valid_list, transform = val_transform) 

In [ ]:
# Create Dataloader for train and validation set
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True) 
valid_loader = DataLoader(valid_data, batch_size = batch_size, shuffle=True)

In [ ]:
print(len(train_loader.dataset))
print(len(train_loader))  # each batch size
print(type(train_loader)) 
print(len(valid_loader.dataset)) 

In [ ]:
# Train and validation loader in a dict
train_val_loader = {"train": train_loader, "val": valid_loader}

In [ ]:
dataset_sizes = {j: len(train_val_loader[j].dataset) for j in ["train", "val"]} 

In [ ]:
dataset_sizes

In [ ]:
# use gpu if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## Transfer Learning and Fintetuning 

Based on the Pytorch Tutorial: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html 

# **Linformer**

In [ ]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)

# Visual Transformer

In [ ]:
"""
train_model_dir = "../input/en-pretrained/efficient_net_b0_finetune.pt"
teacher = EfficientNet.from_name("efficientnet-b0").to(device)  # not trained weights

# recreate architecture as for the training model
num_feature = teacher._fc.in_features  # of the final layer
teacher._fc = nn.Linear(num_feature, 5) # output-layer to 5
teacher = teacher.to(device) 
teacher.load_state_dict(torch.load(train_model_dir, map_location='cuda:0')) 
teacher.eval()
"""

In [ ]:

model = timm.create_model('vit_base_patch16_224', pretrained=True) 
model.head = nn.Linear(768,5)
model = model.to(device)
model.eval()


"""
model = DistillableViT(
    dim=128,
    image_size=224,
    patch_size=32,
    num_classes=5,
    channels=3,
    depth=6,
    heads=8,
    dropout = 0.1,
    emb_dropout = 0.1,
    mlp_dim = 2048
).to(device)

distiller = DistillWrapper(
    student = model,
    teacher = teacher,
    temperature = 3,           # temperature of distillation
    alpha = 0.5                # trade between main loss and distillation loss
)

"""

model

# Training

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [ ]:
best_model_wts = copy.deepcopy(model.state_dict())
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)
        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in valid_loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)
            
    scheduler.step()

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )
    
model.load_state_dict(best_model_wts)

In [ ]:
# Save trained model in output/kaggle/working
path = "./ViT_f.pt"

# Save model
torch.save(model.state_dict(), path) 

## Inference

Save model and load it for inference (support from here: https://pytorch.org/tutorials/beginner/saving_loading_models.html, https://pytorch.org/tutorials/beginner/saving_loading_models.html) 
https://discuss.pytorch.org/t/how-to-use-train-model-for-predict-unseen-data/81689

In [ ]:
# load model 
train_model_dir = "../input/efficientnet-model/efficient_net.pt"
model = EfficientNet.from_name("efficientnet-b3").to(device)  # not trained weights
train_model_dir = "../input/efficientnet-model/efficient_net_b5_finetune.pt"
model = EfficientNet.from_name("efficientnet-b5").to(device)  # not trained weights

# recreate architecture as for the training model
num_feature = model._fc.in_features  # of the final layer
model._fc = nn.Linear(num_feature, 5) # output-layer to 5
model = model.to(device) 

In [ ]:
model.load_state_dict(torch.load(train_model_dir)) 
model.eval()

In [ ]:
# Load test image sample
test_df = pd.read_csv(dataset_dir + "sample_submission.csv")   

# Add to column Image_ID the image path in dataframe 
test_df["path"] = dataset_dir + "test_images/" + test_df["image_id"] 

# Rearrange column order
test_df = test_df[["image_id", "path", "label"]]

In [ ]:
test_df.head()

In [ ]:
test = MyDataset(test_df, val_transform) 

In [ ]:
testloader = DataLoader(test, batch_size = 4, shuffle=False, num_workers=4)

### Feed model test image

Based on this pytorch Tutorial section: "Test the network on the test data" https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [ ]:
d = [2, 3] 
s = [2, 4,2]
d.extend(s)
d

In [ ]:
pred_labels = [] 
for data in testloader: 
    img, label = data  
    img = img.to(device) 
    label = label.to(device)
    
    outputs = model(img) # predict output 
    
    _, predicted = torch.max(outputs, 1) 
    for each in predicted:  # when more/parallel outputs
        pred_labels.append(each.item())

In [ ]:
# Create Submission file with predicted value
submission = pd.DataFrame({"image_id": test_df["image_id"], "label": pred_labels})
submission.head()

In [ ]:
# Save submission file
submission.to_csv('submission.csv', index=False)